In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the dataset from Google Drive
import pandas as pd

dataset_path = '/content/drive/MyDrive/2023SummerResearch/SPY.csv'  # Update the path according to your dataset location
df = pd.read_csv(dataset_path)

In [ ]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-10,256.260010,259.160004,255.500000,258.880005,240.387314,96823900
1,2019-01-11,257.679993,259.010010,257.029999,258.980011,240.480194,73858100
2,2019-01-14,256.859985,258.299988,256.410004,257.399994,239.013062,70908200
3,2019-01-15,257.820007,260.700012,257.809998,260.350006,241.752304,85208300
4,2019-01-16,260.829987,261.970001,260.600006,260.980011,242.337341,77636700
...,...,...,...,...,...,...,...
775,2022-02-07,449.510010,450.989990,445.850006,447.260010,436.823853,84472900
776,2022-02-08,446.730011,451.920013,445.220001,450.940002,440.417999,81012000
777,2022-02-09,455.220001,457.880005,455.010010,457.540009,446.863983,92589900
778,2022-02-10,451.339996,457.709991,447.200012,449.320007,438.835785,140103700


In [ ]:
# Step 1: Prepare the data

#set the DateTime index for the dataframe
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)


In [ ]:
# Step 2: Define the moving average periods
short_window = 50
long_window = 200

In [ ]:
# Step 3: Calculate the moving averages
df['Short_MA'] = df['Close'].rolling(window=short_window).mean()
df['Long_MA'] = df['Close'].rolling(window=long_window).mean()

In [ ]:

# Generate trading signals
df['Signal'] = 0
df.loc[df.index[short_window:], 'Signal'] = (df['Short_MA'][short_window:] > df['Long_MA'][short_window:]).astype(int)
df.loc[df.index[short_window:], 'Signal'] -= (df['Short_MA'][short_window:] < df['Long_MA'][short_window:]).astype(int)



In [ ]:
# Step 5: Implement the strategy
#  a variable named 'portfolio' representing your portfolio holdings and cash balance
portfolio = 0  # Number of shares held
cash = 10000  # Initial cash balance
buy_cost = 0  # Cost of buying shares
sell_cost = 0  # Proceeds from selling shares

for i in range(short_window, len(df)):
    if df['Signal'][i] == 1 and portfolio == 0:  # Buy signal
        portfolio = cash / df['Close'][i]  # Invest all available cash in buying shares
        buy_cost = portfolio * df['Close'][i]  # Update the buy cost
        cash = 0  # Set cash balance to zero
    elif df['Signal'][i] == -1 and portfolio > 0:  # Sell signal
        cash = portfolio * df['Close'][i]  # Sell all shares and get the proceeds
        sell_cost = portfolio * df['Close'][i]  # Update the sell cost
        portfolio = 0  # Set portfolio holdings to zero



In [ ]:
# Step 6: Evaluate the strategy
# Assuming you want to print the portfolio value and total return
portfolio_value = cash if portfolio == 0 else portfolio * df['Close'][-1]
total_return = (portfolio_value + sell_cost - buy_cost) / (buy_cost + sell_cost)

print('Portfolio Value:', portfolio_value)
print('Total Return:', total_return)

Portfolio Value: 12204.380262146622
Total Return: 0.7005216362281055
